# Evaluating LLMs on a Content Generation Task

* docs: https://docs.evidentlyai.com/introduction
* repo: https://github.com/evidentlyai/evidently/

In [ ]:
#! pip install --upgrade "openai"

In [ ]:
import openai

In [ ]:
from openai import OpenAI
import pandas as pd

In [ ]:
my_topics = [
    "testing in AI engineering is as important as in development",
    "CI/CD is applicable in AI",
    "Collaboration of subject matter experts and AI engineers improves product",
    "Start LLM apps development from test cases generation",
    "evidently is agreat for LLM testing, use it"
]

In [ ]:
OA_client = OpenAI()

## Basic content generation

In [ ]:
response = OA_client.responses.create(
        model="gpt-4o-mini",
        input=f"Write a short paragraph about {my_topics[0]}"
    )

text = response

In [ ]:
text.output_text

In [ ]:
text.usage.input_tokens, text.usage.output_tokens

In [ ]:
def basic_tweet_generation(topic, model="gpt-3.5-turbo", instructions=""):
    response = OA_client.responses.create(
        instructions=instructions,
        model=model,
        input=f"Write a short paragraph about {topic}"
    )

    text = response.output_text
    return text

In [ ]:
basic_tweet_generation(my_topics[-1])

## Tracing and Evaluation

In [ ]:
#! pip install tracely

In [ ]:
#! pip install evidently

In [ ]:
from evidently.ui.workspace import Workspace #CloudWorkspace #, RemoteWorkspace

from evidently import Dataset, DataDefinition, Report
from evidently.descriptors import *
from evidently.presets import TextEvals
from evidently.llm.templates import BinaryClassificationPromptTemplate

In [ ]:
from tracely import init_tracing, get_info
from tracely import trace_event, get_current_span

In [ ]:
client = Workspace(path='workspace')
#client = CloudWorkspace()

In [ ]:
project = client.create_project(name="Content evals")
#project = client.create_project("Content Generation: evals & optimization",  
#                                org_id = "ORG ID HERE")

In [ ]:
init_tracing(
    address="http://localhost:8000/",
    project_id=project.id, 
    export_name="basic content generation",
    as_global=False,
)

In [ ]:
@trace_event()
def tweet_generation(topic, model="gpt-3.5-turbo", instructions=""):
    response = OA_client.responses.create(
        instructions=instructions,
        model=model,
        input=f"Write a short paragraph about {topic}"
    )

    span = get_current_span()
    span.update_usage(
        tokens={
            "input": response.usage.input_tokens,
            "output": response.usage.output_tokens,
        }
    )

    text = response.output_text
    return text

In [ ]:
basic_tweets = [tweet_generation(topic, model="gpt-3.5-turbo", instructions="") for topic in my_topics]

In [ ]:
basic_tweets

In [ ]:
get_info()

In [ ]:
dataset_id = get_info()['export_id']
dataset = client.load_dataset(dataset_id)

In [ ]:
dataset.data_definition

In [ ]:
dataset.as_dataframe()

In [ ]:
tweet_quality = BinaryClassificationPromptTemplate(
    pre_messages = [("system","You are evaluating the quality of tweets")],
    criteria="""
        Text is ENGAGING if it meets at least one of the following:
            •        Contains a strong hook (e.g. question, surprise, bold statement)
            •        Uses emotion, humor, or opinion
            •        Encourages interaction (calls to action, second-person voice like “you”)
            •        Demonstrates personality or a distinct tone
            •        Includes vivid language, metaphors, or emojis
            •        Sparks curiosity or gives a new insight

        Text is NEUTRAL if it:
            •        Merely states a fact or observation without emotion or opinion
            •        Lacks clear personality, tone, or call to action
            •        Uses generic language with no rhetorical style
            •        Reads like an internal note, report, or placeholder
        """,
    target_category="ENGAGING",
    non_target_category="NEUTRAL",
    uncertainty="non_target",
    include_reasoning=True,
)

In [ ]:
descriptors = [
    TextLength("tweet_generation.result", alias="Length"),
    Sentiment("tweet_generation.result", alias="Sentiment"),
    LLMEval("tweet_generation.result", template=tweet_quality,
           provider="openai", model="gpt-4o-mini",
           alias="Tweet quality")
]

In [ ]:
dataset.add_descriptors(descriptors=descriptors)

In [ ]:
dataset.as_dataframe()

In [ ]:
report = Report(
    metrics=[TextEvals()]
)

In [ ]:
basic_tweets_eval = report.run(dataset, tags=["gpt-3.5-turbo", "basic generation"])

In [ ]:
basic_tweets_eval

In [ ]:
client.add_run(project.id, basic_tweets_eval, include_data=True)

## Prompt Optimization

In [ ]:
from evidently.llm.optimization import PromptOptimizer, PromptExecutionLog, Params

In [ ]:
def run_prompt(generation_prompt: str, context) -> pd.Series:
    """generate engaging tweets"""
    my_topics = [
        "testing in AI engineering is as important as in development",
        "CI/CD is applicable in AI",
        "Collaboration of subject matter experts and AI engineers improves product",
        "Start LLM apps development from test cases generation",
        "evidently is a great tool for LLM testing"
    ]
    tweets = [basic_tweet_generation(topic, model="gpt-3.5-turbo", instructions=generation_prompt) for topic in my_topics]
    return pd.Series(tweets)

In [ ]:
judge = LLMEval("tweet_generation.result", template=tweet_quality,
                provider="openai", model="gpt-4o-mini", alias="Tweet quality")

In [ ]:
optimizer = PromptOptimizer("tweet_gen_example", strategy="feedback", verbose=True)
await optimizer.arun(run_prompt, scorer=judge, base_prompt="You are tweet generator", repetitions=2)

In [ ]:
print(optimizer.best_prompt())

In [ ]:
optimizer.print_stats()

In [ ]:
init_tracing(
    address="http://localhost:8000/",
    project_id=project.id, # Project ID from Evidently Cloud
    export_name="optimised content generation",
    as_global=False,
)

In [ ]:
better_tweets = [tweet_generation(topic, model="gpt-4o-mini", instructions=optimizer.best_prompt()) for topic in my_topics]

In [ ]:
get_info()

In [ ]:
dataset_id = get_info()['export_id']
optimized_dataset = client.load_dataset(dataset_id)

In [ ]:
optimized_dataset.add_descriptors(descriptors=descriptors)

In [ ]:
optimized_dataset.as_dataframe()

In [ ]:
optimized_tweets_eval = report.run(optimized_dataset, tags=["gpt-4o-mini", "optimized generation"])

In [ ]:
optimized_tweets_eval

In [ ]:
client.add_run(project.id, optimized_tweets_eval, include_data=True)